In [1]:
library(tibble)

In [2]:
create_df <- function() {
    df <- read.csv("fifo.csv", header = TRUE, sep = ",")
    #df$tally_q <- as.integer(df$tally_q)
    # df$sale <- as.integer(df$sale)
    # df$date <- as.Date(df$date, origin = "1899-12-30")

    df$cf_quan <- as.integer(rep(0, nrow(df)))
    df$cf_pps <- as.numeric(rep(0, nrow(df)))
    df$sell_quan <- as.integer(rep(0, nrow(df)))
    df$sell_price <- as.numeric(rep(0, nrow(df)))
    df$sell_proceeds <-as.numeric(rep(0, nrow(df)))
    df$profit_loss <- as.numeric(rep(0, nrow(df)))
    return(df)   # needed to return all the changes made to df
    }
df <- create_df()  # fct call

In [3]:


carryforward_test <- function(buy_quan) {
    
    cat("\t\t", "Howdy from the carryforward test fct. This fct will combine the cf_quan, ", df[i, 'cf_quan'], " with the quan from buy row following the sell row: ", df[i+2, 'quan'], "\n")
    buy_quan <- df[i, 'cf_quan'] + df[i+2, 'quan']
    cat("\t\t", "The new sell_quan is: ", buy_quan, "\n")
    
    return(buy_quan)
          
    }

In [4]:
sell_carryforward <- function(df, buy_quan) {
    

    unsold_carryforward <- df[i , 'cf_quan']
    
cat("temp cat: unsold_carryforward is: ", unsold_carryforward, " Imported buy_quan is: ", buy_quan, "\n")
    
    
#    if(df[i, 'cf_quan'] > 0) {
    if(unsold_carryforward > 0) {
        
        cat("This sale includes a c/f ", unsold_carryforward, "\n" )
        
        #testing program variables
        cat("Testing var values in child fct: ", buy_quan, ", ", sell_quan, ", ",  
            buy_ct,  ", ",   sell_ct,  ", ",  buy_index, ", ",   sell_index,  ", ",  
            sell_price,  ", ",  sell_proceeds, "\n")       
            
#             return()
#             }
      
        temp <- df[i, ]
        # Next, make changes (allocate the remainder values) to the new row before combining new row with df
              
        #    temp[i, 'type'] <- "Buy1"           # to prevent unwanted advances in the loop
            
        
       
        df[i, 'type'] <- "SOLD"
        df[i, 'quan'] <- unsold_carryforward
        df[i, 'basis'] <- unsold_carryforward * df[i, 'pps']
        df[i, 'tally_q'] <- 0
        df[i, 'tally_b'] <- 0
        df[i+1, 'cf_quan'] <- 0
        df[i+1, 'cf_pps'] <-  0
        df[i, 'sell_quan'] <- unsold_carryforward
        df[i, 'sell_price'] <- sell_price       
        # df[i, 'sell_proceeds'] <- sell_price * unsold_carryforward
        # df[i, 'profit_loss'] <- df[i,'sell_proceeds'] - df[i,'basis']
        
        # update the Sell row
        df[i+1, 'quan'] <- sell_quan - unsold_carryforward
        df[i+1, 'basis'] <- df[i+1, 'quan'] * df[i+1, 'pps']
        
    df_test <<- df
    write.csv(df, "output1.csv", row.names = FALSE)
    
    return(df)        
    } else if(unsold_carryforward <= 0) {
        cat("There's an issue with unsold_carryforward in the sell_carryforward fct. ", unsold_carryforward, "\n")
        }
} # end fct
    

In [5]:
# This will reverse (sell bottom / buy top ) the transaction   The second half

# This fct get called from:   check_sell <- function(df) 


reverse_sell <- function(df) {
    
    # This all involves allocating the Sell of 1000 from Row 3.
    # The Sell takes the previous 50 c/f, then will take 950 from the Buy on Row 4
    # The buy_index, sell_index come 3 and 2, 
    # A new row will be created in Row 5.  This will be the C/F from the 50 remaining from Buy in Row 4 
    # The new row will be created (copied from the Buy on row 4) and inserted into row 5.   
    # All remaining unapplied Sell (950) will get applied to the Buy on Row 4.  
    # The c/f of 50 on Row 5 will need most variables updated

    
    
   
    
    
    df <- df
    cat("\n\n\n\n\t\t\t\t", "In the reverse_sell fct.  Checking buy_index, sell_index, buy_quan, sell_quan, and i values ", "\n")
    
    cat(buy_index, "\t", sell_index,"\t", buy_quan, "\t", sell_quan, "\t", i, "\n") 
    
    
             # make some updates to row 4 before creating Row 5
            
             # Note: i comes in with a value = 4    , buy_quan = 1000, sell_ quan = 950 , buy_index == 3, sell_index = 2
    
            # df[i, 'type'] <- "SOLD"
    
            sell_first_quan <- buy_quan - sell_quan
            sell_second <- sell_quan
    
            # get some values before anything
            sell_pps <- df[i, 'pps']     # from for-loop at 2 
            sell_quan2 <- df[i, 'quan']
            buy2_pps <- df[i+1, 'pps']
            buy2_quan <- df[i+1, 'quan']
            old_cf <- abs(df[i, 'quan'])
            
            # set some values before row copy
            df[i, 'quan'] <- sell_quan 
            df[i, 'tally_q'] <- sell_quan 
            df[i, 'tally_b'] <- df[i, 'tally_q'] * df[i, 'pps']
            # create a Buy row from the df
            temp <- df[i , ]   
            # Add row       
            df <- add_row(df, temp, .after = i)
    
    # ----------------------------------------------------------------------------------------------------
            # row 3
    
            # set more values in row 3 
            df[i, 'cf_quan'] <- sell_quan - buy_quan    # reverse the value in cf_quan 
            df[i, 'sell_quan'] <-  buy_quan - sell_quan
            df[i, 'tally_q'] <-  sell_quan
            df[i, 'sale'] <- buy_quan - sell_quan
            df[i, 'sell_price'] <- df[i, 'pps']
            df[i, 'cf_pps'] <- df[i-1, 'cf_pps']
            df[i, 'sell_proceeds'] <- df[i, 'sell_quan'] * df[i, 'sell_price'] 
            df[i, 'profit_loss'] <- df[i,'sell_proceeds'] -( df[i, 'cf_pps'] * df[i, 'sell_quan'])
    
    
    # =================================================================================================================
 
    cat("info cat: ", buy_quan,"\t",  sell_quan,"\t", buy_ct, "\t", sell_ct,"\t",  buy_index,"\t",  sell_index,"\t",  sell_price,"\t",  sell_proceeds, "\n") 
    
    # =================================================================================================================
    
    
           # row 4  - na
    
     
    # =================================================================================================================
    
    
            # new row ?
    
    
            df[i+1, 'type'] <- "Sold"
            df[i, 'sale'] <- 0 
    
            # copy another row
            temp <- df[i+1 , ]   
            # Add row       
            df <- add_row(df, temp, .after = i)
    
        # =================================================================================================================
            
            # Update new / proceeding row  ( row 5 )
            
            df[i+1, 'type'] <- "Sold"
            df[i+1, 'quan'] <- sell_quan
            df[i+1, 'sale'] <- sell_quan
            df[i+1, 'tally_q'] <- 0
            df[i+1, 'sell_quan'] <- sell_quan
            df[i+1, 'sell_price'] <- df[i+1, 'pps']
            df[i+1, 'sell_proceeds'] <- df[i+1, 'sell_quan'] * df[i+1, 'sell_price']
    
    
            # row 6
            df[i+2, 'type'] <- "CF"
            df[i+2, 'tally_q'] <- buy_quan - sell_quan
    
#             df[i+1, 'basis'] <- sell_quan * df[i, 'pps']
#             df[i, 'quan'] <- buy_quan
#             df[i+1, 'type'] <- "Sold"
#             df[i+1, 'sell_quan'] <- buy_quan
#             df[i+1, 'cf_pps'] <- df[i+1, 'pps']
#             df[i+1, 'sell_price'] <- df[i+2, 'pps']
        
            
     
            
    
    return(df)
    
    }

In [6]:
# When there is a pair of but/sell rows

buy_sell_easy1 <- function(df) {
   
    df <- df
    # allocate the unsold Buy quan (this will become a new row in df )
    unsold_carryforward <- buy_quan - sell_quan
    # sold_non_carryforward <- sell_quan - df_carryforward
    
    
    cat("\t\t", "top of buy_sell_easy1 fct. The buy_quan/sell_quan values are: ", buy_quan, "\t", sell_quan, " and value of i: ", i,  "\n")
    
    # test for type of sale tranaction
    if(abs(sell_quan) == buy_quan) {
        sell_quan <- abs(df[i+1, 'quan'])
       
        
       cat("This will be a simple wash sale with no c/f", "\n")
       # one possibility of eliminating these rows is to zero the quans and set a flag
        # df[i, 'sell_q' ] <- sell_quan
        # df[i, 'sell_p'] <- df[i+1, 'pps'] * sell_quan
        # df[i +1, 'type'] <- "Sold"
        # df[i +1, 'index'] <- i
        # sell_quan <- 0
        # sell_index <- 0
        # cat("Clean sale. No C/F from sales condition type 1 ", "\n")
        return(df)       
        
    }
    
    else if(abs(sell_quan) > buy_quan && df[i, 'cf_quan'] == 0) {
        
        sell_quan <- abs(df[i+1, 'quan'])       
        
        cat("This will be a simple wash sale with no c/f", "\n")
        return(df)
        }
# includes fct call  
     else if(abs(sell_quan) > buy_quan && buy_index == sell_index) {
        
         cat("This will NOT be a simple wash sale " , "\n")
         
      #   cat("This is a temp cat: to determine if the changed sell_quan from else if above initiates this else if stmt: ", sell_quan, "\n")
         
         buy_quan <- carryforward_test(buy_quan) 
         
         cat("Still in big_easy fct, the else-if that calls the carryforward test fct. Return buy_quan is: ", buy_quan, "\n")
                 
        cat("troublesome vars involving remaindes and carryforwards", "\n")
        

# fct call
         df <- sell_carryforward(df,buy_quan)
              
         return(df)         # Dont return.  Hopefully this will go down to the next else-if
        }
         
         
         
    else if(abs(sell_quan) < buy_quan) {  
        
       
    #    sell_quan <- abs(df[i+1, 'quan'])
        cat("troublesome vars involving remaindes and carryforwards", "\n")
        
   # First things first, determine if this sale will include a preceeding c/f amount
        
        if(df[i, 'cf_quan'] > 0) {
            cat("This sale includes a c/f ", unsold_carryforward, "\n" )
            return()
            }
        
  # -----------------------------------------------------------------------------      
                   
            # create a row, then update some values on that row
            temp <- df[i, ]
            temp[i, 'quan'] <- unsold_carryforward
            temp[i, 'basis'] <- temp[i, 'pps'] * unsold_carryforward
            temp[i, 'index'] <- buy_index + 1
        # Add the new row to the df,      
        df <- add_row(df, temp, .before = i+1)
        
# -------------------------------------------------------------------------------------     
        
        # Next, make changes (allocate the remainder values) to the new row 
        df[i, 'type'] <- "SOLD"
        df[i, 'quan'] <- sell_quan 
        df[i, 'basis'] <- sell_quan * df[i, 'pps']
        df[i, 'tally_q'] <- sell_quan - sell_quan - unsold_carryforward
        df[i, 'tally_b'] <- df[i, 'tally_q'] * df[i, 'pps']
        df[i+1, 'cf_quan'] <- unsold_carryforward
        df[i+1, 'cf_pps'] <-  df[i, 'pps']
        df[i, 'sell_quan'] <- sell_quan
        df[i, 'sell_price'] <- sell_price
        df[i, 'profit_loss'] <- df[i, 'basis'] - sell_proceeds
        df[i, 'sell_proceeds'] <- sell_price * sell_quan
            
# There is the remaining portion of Sell (950) 


        return(df)          
                
    
         
        # when the buys row i has quanity < sell quan, need carry values forward to next row,
         # And mo
       } else if(sell_ct - buy_ct == 0 && df[i, 'quan']  > df[i+1, 'quan'] ) {
        
        # fct call
        
        # Moved the code for this else if to fct().  Works
   #     df <- partial(df) 
        return(df)      
        }
     
        


    }  # fct
    
    

In [7]:
buy_sell_easy <- function(df) {
    
    cat("In buy_sell_easy fct.  Testing var seethru.  buy_quan and sell quan are: ", buy_quan, " and ", sell_quan, " buy_index and sell_index are: ", buy_index, " and ", sell_index, "\n")  
    df <- buy_sell_easy1(df)
    return(df)
    }

 buy_sell_advanced <- function(df) {
     df <- df
    cat("In buy_sell_Advanced fct.  buy_quan and sell quan are: ", buy_quan, " and ", sell_quan, " buy_index and sell_index are: ", buy_index, " and ", sell_index, "\n") 
     return(df)
    } 

buy_sell_partial <- function(df) {
    df <- df
    cat("In buy_sell_partial fct  buy_quan and sell quan are: ", buy_quan, " and ", sell_quan, " buy_index and sell_index are: ", buy_index, " and ", sell_index, "\n")  
    return(df)
    }

check_sell <- function(df) {
    df <- df
     cat("In check_sell fct buy_quan and sell quan are: ", buy_quan, " and ", sell_quan, " buy_index and sell_index are: ", buy_index, " and ", sell_index, "\n") 
    if(df[i+1, 'type'] == "Buy") {
        cat("Execute the reverse_sell fct ", "\n")
 # fct call        
      #  df <- reverse_sell(df)
        return(df)
        }
    }


## <font color = yellow> Main

In [8]:
#  Move down the df from row to row 

df_test<- data.frame()
df_sales <- data.frame()

for(i in 1:nrow(df[1:2,])) {   # hold to 6 itertions
    
    
    
    index_row <- i
    
    if(df[i, 'type'] == "Buy" && df[i+1, 'type'] == "Sell") {
        buy_quan <- df[i, 'quan']
        sell_quan <- df[i+1, 'quan']    
        buy_ct <- df[i, 'buy_ct']
        
        sell_ct <- df[i+1, 'buy_ct'] 
        buy_index <- df[i, 'index']
        
        sell_index <- df[i+1, 'index']
        sell_price <- df[i +1, 'pps']
        sell_proceeds <- sell_price * sell_quan
        
   }else if(df[i, 'type'] == "Sell" && df[i+1, 'type'] == "Buy") {
        buy_quan <- df[i+1, 'quan']
        sell_quan <- df[i, 'quan']   
        
        buy_ct <- df[i+1, 'buy_ct']
        sell_ct <- df[i, 'buy_ct'] 
        
        buy_index <- df[i+1, 'index']
        sell_index <- df[i, 'index']
        sell_price <- df[i, 'pps']
        sell_proceeds <- sell_price * sell_quan
        }
    
    

    cat("\n", "Loop row number is: ", i, "\n")
    
    cat("temp cat: buy_index and sell_index are ", buy_index, "\t", sell_index, "\n")
           

    if( df[i, 'type'] ==  "Buy") {

        cat("condition 1: ", df[i, 'type'], "\n")
        # simple buy/sell rows 
        if(df[i+1, 'type'] == "Sell") {
            df <- buy_sell_easy(df)
                        
            #  Stacked Buy rows
            }else if (df[i+1, 'type'] == "Buy") {
                buy_sell_advanced(df) 
            }
     }else if (df[i, 'type'] == "Part Sell") {

        cat("condition 2: ", df[i, 'type'], "\n")
        buy_sell_partial(df)

    }else if ( df[i, 'type'] == "Sell") {

    # increment to next row from top of loop 

    cat("condition 3: ", df[i, 'type'], "\n")
        df <-check_sell(df)
        # return(df)


    } else {

        cat("condition 4 - Unrecognized type in row: ", i, " with 'type': ",  df[i, 'type'], "buy_quan and sell quan are: ", buy_quan, " and ", sell_quan, " buy_index and sell_index are: ", buy_index, " and ", sell_index, "\n") 
        }
 
 } # for   
    

    



 Loop row number is:  1 
temp cat: buy_index and sell_index are  1 	 2 
condition 1:  Buy 
In buy_sell_easy fct.  Testing var seethru.  buy_quan and sell quan are:  1050  and  1000  buy_index and sell_index are:  1  and  2 
		 top of buy_sell_easy1 fct. The buy_quan/sell_quan values are:  1050 	 1000  and value of i:  1 
troublesome vars involving remaindes and carryforwards 

 Loop row number is:  2 
temp cat: buy_index and sell_index are  2 	 2 
condition 1:  Buy 
In buy_sell_easy fct.  Testing var seethru.  buy_quan and sell quan are:  50  and  1000  buy_index and sell_index are:  2  and  2 
		 top of buy_sell_easy1 fct. The buy_quan/sell_quan values are:  50 	 1000  and value of i:  2 
This will NOT be a simple wash sale  
		 Howdy from the carryforward test fct. This fct will combine the cf_quan,  50  with the quan from buy row following the sell row:  1000 
		 The new sell_quan is:  1050 
Still in big_easy fct, the else-if that calls the carryforward test fct. Return buy_quan is

ERROR: Error in if (df[i, "type"] == "Buy" && df[i + 1, "type"] == "Sell") {: missing value where TRUE/FALSE needed


In [ ]:
head(df, 10)

In [ ]:
tail(df,3)

In [ ]:
temp

In [ ]:
temp <- df[2, ]
df <- add_row(df, temp, .before = 3)
head(df)
str(df)
tail(df)
nrow(df)

In [ ]:
head(df)